In [1]:
from http.client import MULTI_STATUS
import re
import pandas as pd
import numpy as np
import re

import numpy as np

def load_dfs():
    dfs = []
    for i in range(4):
        df = pd.read_csv(f'overall_{i+1}.csv', index_col=[0,1], header=[0,1])

        df.columns = pd.MultiIndex.from_tuples([tuple(i[0].split(': ') + [i[1]]) for i in df.columns], names=['nn', 'metric', 'how'])

        display(df)

        df.columns = pd.MultiIndex.from_tuples([tuple((col[0][0:-7], col[0][-3:], col[1][1:3]))for col in df.columns], names=['source', 'th', 'dns type'])

        df = df.sort_index(level=[0,1,2], ascending=[True, True, False], axis=1)

        df = df[[col for col in df.columns if col[0].find('no-renew') >= 0]]

        s = 1 - df.loc[('PDay', 'sum')] / (df.loc[('ADay', 'sum')])
        s.name = ('UDay', 'ratio')

        df = pd.concat([df, s.to_frame().T])

        df = df.map(lambda x: '-' if np.isnan(x) else (f'{int(x)}' if x == int(x) else f'{round(x, 2):.2f}') )

        display(df)

        df = df.T.unstack(1)

        # df = df.drop(columns=['first MA', ], level=0)
        df = df.droplevel(1, axis=1)

        cols = [ (col, th) for col in ['UDay', 'TAR', 'MAR', 'first TA', 'TA', 'MA', 'A', 'PDay', 'ADay'] for th in ['1.0', '1.5']]

        df = df[cols]
        dfs.append(df)
    return dfs


In [166]:
from defs import NN

def load_dfs():
    dfs = []
    for nn in NN:
        df = pd.read_csv(f'overall_{nn}.csv', index_col=[0, 1, 2])

        # df = df.unstack(0)

        # df.columns = pd.MultiIndex.from_tuples([tuple(i[0].split(': ') + [i[1]]) for i in df.columns], names=['nn', 'metric', 'how'])

        rgx = r'df \d+ (\w+) no\-renew.*'

        df = df.unstack(2).T
        df.index = pd.MultiIndex.from_tuples([tuple([str(nn), re.sub(rgx, r'\1', i[1]), i[1][-3:]]) for i in df.index], names=['nn', 'malware', 'th_s'])
        df.columns = pd.MultiIndex.from_tuples([tuple(i[0].split(': ') + [i[1]]) for i in df.columns], names=['pt', 'metric', 'how'])

        df = df.swaplevel(1, 0, axis=1)
        df = df.sort_index(level=0, axis=1)
        df = df.droplevel(2, axis=1)
        df = df.sort_index(level=[0,1], axis=0)

        dfs.append(df)
        pass

    df = pd.concat(dfs)

        # print(df.to_markdown(), end='\n\n')


    # df = df.drop(columns=['MA', 'PDay', 'TA', 'TAR'], level=0)
    # df = df.drop(columns=[list(zip('A', ['gr_pos', 'gok_pos']))], level=0)


    df = df.unstack(-1).swaplevel(0, 2, axis=1).swaplevel(1, 2, axis=1).sort_index(level=[0,1,2], ascending=[True,True,False], axis=1)

    return df.swaplevel(0,1,axis=0).sort_index(level=[0,1])

_ = [ print(f'{str(col) + (', ' if (i+1) % 4 != 0 else ',\n'):>25}', end='') for i, col in enumerate(list(load_dfs().columns.droplevel(0).drop_duplicates()))]

load_dfs()

        ('A', 'gr_pos'),             ('A', 'gr'),        ('A', 'gok_pos'),            ('A', 'gok'),
       ('A', 'gnx_pos'),            ('A', 'gnx'),      ('ADay', 'gr_pos'),          ('ADay', 'gr'),
    ('ADay', 'gok_pos'),         ('ADay', 'gok'),     ('ADay', 'gnx_pos'),         ('ADay', 'gnx'),
      ('MA', 'gok_pos'),       ('MA', 'gnx_pos'),     ('PDay', 'gok_pos'),     ('PDay', 'gnx_pos'),
      ('TA', 'gok_pos'),       ('TA', 'gnx_pos'), ('first MA', 'gok_pos'), ('first MA', 'gnx_pos'),
('first TA', 'gok_pos'), ('first TA', 'gnx_pos'), 

th_s           1.0                                                         \
metric           A                                            ADay          
pt          gr_pos      gr gok_pos     gok gnx_pos     gnx  gr_pos     gr   
malware nn                                                                  
caphaw  1   6117.0  6181.0   273.0   369.0  6100.0  6180.0  6117.0  810.0   
        2   4932.0  6181.0   136.0   369.0  4924.0  6180.0  4932.0  810.0   
        3   5220.0  6181.0   192.0   369.0  5204.0  6180.0  5220.0  810.0   
        4   5261.0  6181.0   153.0   369.0  5244.0  6180.0  5261.0  810.0   
simda   1    393.0   393.0  2058.0  2058.0   384.0   384.0   393.0   85.0   
        2    393.0   393.0  2058.0  2058.0   384.0   384.0   393.0   85.0   
        3    393.0   393.0  2058.0  2058.0   384.0   384.0   393.0   85.0   
        4    393.0   393.0  2058.0  2058.0   384.0   384.0   393.0   85.0   
unknown 1     88.0    97.0   192.0   201.0    64.0    64.0    88.0   66.0   
        2     88.0    97.0   192.0   201.0    64.0    64.0    88.0   66.0   
        3     88.0    97.0   192.0   201.0    64.0    64.0    88.0   66.0   
        4     89.0    97.0   193.0   201.0    64.0    64.0    89.0   66.0   
zbot    1   6504.0  6504.0  7216.0  7216.0  3832.0  3832.0  6504.0  865.0   
        2   6504.0  6504.0  7216.0  7216.0  3832.0  3832.0  6504.0  865.0   
        3   6504.0  6504.0  7216.0  7216.0  3832.0  3832.0  6504.0  865.0   
        4   6504.0  6504.0  7216.0  7216.0  3832.0  3832.0  6504.0  865.0   

th_s                        ...     1.5                                  \
metric                      ...      MA            PDay              TA   
pt         gok_pos     gok  ... gok_pos gnx_pos gok_pos gnx_pos gok_pos   
malware nn                  ...                                           
caphaw  1    273.0   295.0  ...   369.0  2142.0     0.0   770.0     0.0   
        2    136.0   295.0  ...   369.0  2848.0     0.0   734.0     0.0   
        3    192.0   295.0  ...   369.0  2001.0     0.0   745.0     0.0   
        4    153.0   295.0  ...   369.0  4486.0     0.0   564.0     0.0   
simda   1   2058.0   304.0  ...  2058.0    55.0     0.0    67.0     0.0   
        2   2058.0   304.0  ...  2027.0    21.0    28.0    69.0    31.0   
        3   2058.0   304.0  ...  2058.0    15.0     0.0    71.0     0.0   
        4   2058.0   304.0  ...  2058.0    62.0     0.0    67.0     0.0   
unknown 1    192.0   104.0  ...   201.0    24.0     0.0    33.0     0.0   
        2    192.0   104.0  ...   141.0    14.0    53.0    43.0    60.0   
        3    192.0   104.0  ...   201.0     2.0     0.0    41.0     0.0   
        4    193.0   104.0  ...   201.0    24.0     0.0    33.0     0.0   
zbot    1   7216.0  1002.0  ...  7204.0  3416.0     7.0   338.0    12.0   
        2   7216.0  1002.0  ...  7160.0  2482.0    21.0   795.0    56.0   
        3   7216.0  1002.0  ...  7173.0  1771.0    18.0   931.0    43.0   
        4   7216.0  1002.0  ...  7212.0  3629.0     4.0   158.0     4.0   

th_s                                                        
metric              first MA            first TA            
pt         gnx_pos   gok_pos   gnx_pos   gok_pos   gnx_pos  
malware nn                                                  
caphaw  1   4038.0  4.298305  2.852252       NaN  1.616883  
        2   3332.0  4.298305  2.755043       NaN  1.502725  
        3   4179.0  4.298305  3.003195       NaN  1.544966  
        4   1694.0  4.298305  1.843710       NaN  1.851064  
simda   1    329.0  1.190789  4.125000       NaN  1.940299  
        2    363.0  1.190789  4.800000  2.785714  1.811594  
        3    369.0  1.190789  5.000000       NaN  1.901408  
        4    322.0  1.190789  4.138889       NaN  1.940299  
unknown 1     40.0  1.067308  1.411765       NaN  1.000000  
        2     50.0  1.387755  2.000000  1.528302  1.000000  
        3     62.0  1.067308  1.000000       NaN  1.000000  
        4     40.0  1.067308  1.411765     

In [174]:
import re

import numpy as np

df = load_dfs()

cols = [
        ('A', 'gr_pos'),             ('A', 'gr'),        ('A', 'gok_pos'),            ('A', 'gok'),
       ('A', 'gnx_pos'),            ('A', 'gnx'),      ('ADay', 'gr_pos'),          ('ADay', 'gr'),
    ('ADay', 'gok_pos'),         ('ADay', 'gok'),     ('ADay', 'gnx_pos'),         ('ADay', 'gnx'),
      ('MA', 'gok_pos'),       ('MA', 'gnx_pos'),    
      ('PDay', 'gok_pos'),     ('PDay', 'gnx_pos'),
      ('TA', 'gok_pos'),       ('TA', 'gnx_pos'),
    #   ('first MA', 'gok_pos'), ('first MA', 'gnx_pos'),
    ('first TA', 'gok_pos'), ('first TA', 'gnx_pos')
]

df = df[[ (th, col[0], col[1]) for th in ['1.0', '1.5'] for col in cols]]


for th in ['1.0', '1.5']:
    for l in ['gok_pos', 'gnx_pos']:
        df[(th, 'TAR', l)] = df[(th, 'TA', l)] / (df[(th, 'TA', l)] + df[(th, 'MA', l)])
        # df[(th, 'MAR', l)] = df[(th, 'MA', l)] / (df[(th, 'TA', l)] + df[(th, 'MA', l)])
        df[(th, 'TDR', l)] = df[(th, 'PDay', l)] / (df[(th, 'PDay', l)] + df[(th, 'ADay', l)])
        df = df.drop(columns=(th, 'PDay', l))
        df = df.drop(columns=(th, 'MA', l))
        df = df.drop(columns=(th, 'TA', l))


df = df[[col for col in df.columns if not (col[0] == '1.5' and col[1] in ['A', 'ADay'])]]


df1 = df.loc[:, pd.IndexSlice[:, ['A', 'ADay'], :]]
df2 = df.loc[:, [col for col in df.columns if col not in df1.columns]]

with open('/Users/princio/Repo/princio/Paper-IEEEE-Access/_table.tex', 'w') as fp:
    for i, df in enumerate([df1, df2]):
        df = df.map(lambda x: '-' if np.isnan(x) else (f'{int(x)}' if x == int(x) else f'{round(x, 2):.2f}') )
        df = df.sort_index(level=0, axis=1)

        s = df.style
        if i == 1:
            for mw,_ in df.index:
                def highlight_max(x, color):
                    return np.where(x == np.nanmax(x.to_numpy()), f"color: {color}; font-weight: bold;", None)
                def highlight_min(x, color):
                    return np.where(x == np.nanmin(x.to_numpy()), f"color: {color}; font-weight: bold;", None)
                for th in ['1.0','1.5']:
                    s = s.apply(highlight_max, color='ForestGreen', subset=(pd.IndexSlice[mw, :], pd.IndexSlice[th, 'TDR', :]))
                    s = s.apply(highlight_min, color='BrickRed', subset=(pd.IndexSlice[mw, :], pd.IndexSlice[th, 'TDR', :]))

                    s = s.apply(highlight_max, color='ForestGreen', subset=(pd.IndexSlice[mw, :], pd.IndexSlice[th, 'TAR', :]))
                    s = s.apply(highlight_min, color='BrickRed', subset=(pd.IndexSlice[mw, :], pd.IndexSlice[th, 'TAR', :]))
                    pass
                pass

        t = s.to_latex(convert_css=True)
        rgx = re.compile(r'\\cline\{\d-\d+\}', re.MULTILINE)
        t = rgx.sub('', t)
        t = re.sub(r'^\\multirow\[\w*\]\{\d\}\{\*\}\{(\w+)\}', r'\\multicolumn{6}{l}{\1} \\\\\n', t, 0, re.MULTILINE)
        t = re.sub(r'\\multicolumn\{(\d)\}\{\w\}', r'\\multicolumn{\1}{l}', t, 0, re.MULTILINE)
        t = t.replace(r'0.', '.')
        
        t = t.replace('{r}', '{l}')
        t = t.replace('_', '\\_')
        t = t.replace('\\bfseries \\bfseries \\bfseries \\bfseries', '\\bfseries')
        t = t.replace('\\itshape \\itshape \\itshape \\itshape', '\\itshape')
        t = t.replace('\\bfseries \\itshape \\bfseries \\itshape \\bfseries \\itshape \\bfseries \\itshape', '\\bfseries \\itshape')
        fp.write('''
    \\begin{table*}
        \\centering
        \\resizebox*{\\textwidth}{!}{%\n''' + t + '''
        }
        \\caption{Bo ''' + str(1+i) + '''.}
        \\label{tab:results}
    \\end{table*}


    ''')
